In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("app1").getOrCreate()
spark

23/08/01 10:34:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [27]:
csv_file_path="./data/file1.csv"
df = spark.read.format("csv")\
        .option("header","true")\
        .option("inferSchema","true") \
        .load(csv_file_path)

df.printSchema()
df.show()

root
 |-- col1: string (nullable = true)
 |-- col2: string (nullable = true)
 |-- col3: string (nullable = true)
 |-- col4: string (nullable = true)
 |-- age: integer (nullable = true)

+-----+-----+-----+-----+---+
| col1| col2| col3| col4|age|
+-----+-----+-----+-----+---+
|data1|data2|data3|data4| 26|
|data1|data2| null|data4| 99|
|data1|data2| null|data4| 63|
+-----+-----+-----+-----+---+



In [31]:
input_args = {
    "header": "true",
    "inferSchema": "true"
}

df = spark.read.format("csv")
for key,value in input_args.items():
   df =  df.option(key,value)

df = df.load(csv_file_path)

df.printSchema()
df.show()

root
 |-- col1: string (nullable = true)
 |-- col2: string (nullable = true)
 |-- col3: string (nullable = true)
 |-- col4: string (nullable = true)
 |-- age: integer (nullable = true)

+-----+-----+-----+-----+---+
| col1| col2| col3| col4|age|
+-----+-----+-----+-----+---+
|data1|data2|data3|data4| 26|
|data1|data2| null|data4| 99|
|data1|data2| null|data4| 63|
|data1|data2| null|data4|160|
+-----+-----+-----+-----+---+



In [13]:
#### write 
#df.write.save("./ouput/")

df.write.mode("append").save("./output/")

In [32]:
### Null value 
from pyspark.sql.functions import col 

for column in df.columns:
    null_count = df.filter(col(column).isNull()).count()
    if null_count > 0:
        raise ValueError(f"column {column} has {null_count} null values")


ValueError: column col3 has 3 null values

In [18]:
### Uniquenss check 
if df.count() != df.distinct().count():
    raise ValueError("Duplicate rows found in the dataframe")

In [33]:
# Range Check 
age_min = 0 
age_max = 150

invalid_age_count = df.filter((col("age") < age_min) | (col("age") > age_max)).count()
if invalid_age_count > 0:
    raise ValueError(f"{invalid_age_count} rows have invalid age values")

ValueError: 1 rows have invalid age values

In [34]:
### Completeness Check 
required_columns = ["name", "email", "age"]  # Add all required columns here
for column in required_columns:
    null_count = df.filter(col(column).isNull()).count()
    if null_count >0:
        raise ValueError(f"Column {column} has {null_count} null values")

AnalysisException: cannot resolve '`name`' given input columns: [age, col1, col2, col3, col4];;
'Filter isnull('name)
+- Relation[col1#540,col2#541,col3#542,col4#543,age#544] csv
